In [18]:
import nba_api.stats.endpoints as nba
import boto3
import json
from botocore.exceptions import ClientError
from decimal import Decimal



In [19]:

REGION = "us-east-2"
BUCKET_NAME = "netstats-data"
dynamodb = boto3.resource("dynamodb", region_name=REGION)
s3 = boto3.client("s3")
client = boto3.client("dynamodb", region_name=REGION)

def load_json_from_s3(key):
    obj = s3.get_object(Bucket=BUCKET_NAME, Key=key)
    return json.load(obj["Body"], parse_float=Decimal)

In [20]:
touch_data = nba.LeagueDashPtStats(season="2024-25", player_or_team="Player", pt_measure_type = "Possessions", per_mode_simple="PerGame")

In [21]:
touch_data.get_data_frames()[0][["PLAYER_ID", "PLAYER_NAME", "TOUCHES", "TIME_OF_POSS", "AVG_SEC_PER_TOUCH", "AVG_DRIB_PER_TOUCH", "PTS_PER_TOUCH"]]

,PLAYER_ID,PLAYER_NAME,TOUCHES,TIME_OF_POSS,AVG_SEC_PER_TOUCH,AVG_DRIB_PER_TOUCH,PTS_PER_TOUCH
0,1630639,A.J. Lawson,31.0,1.1,2.16,1.34,0.293
1,1631260,AJ Green,29.3,0.9,1.80,0.91,0.253
2,1642358,AJ Johnson,40.3,2.8,4.10,3.67,0.188
3,203932,Aaron Gordon,41.9,2.2,3.14,2.21,0.350
4,1628988,Aaron Holiday,20.3,1.3,3.77,3.57,0.270
...,...,...,...,...,...,...,...
564,203897,Zach LaVine,61.4,4.3,4.21,3.65,0.379
565,1630192,Zeke Nnaji,9.5,0.3,1.60,0.58,0.338
566,1630533,Ziaire Williams,33.5,1.1,2.03,1.25,0.299
567,1629627,Zion Williamson,66.4,4.2,3.79,3.22,0.370


In [27]:
all_season_dicts = {}

seasons = [
    "2014-15", "2015-16", "2016-17", "2017-18", "2018-19",
    "2019-20", "2020-21", "2021-22", "2022-23", "2023-24", "2024-25"
]

for season in seasons:
    print(season)
    # Load the data for that season (replace this with actual call)
    touch_data = nba.LeagueDashPtStats(season=season, player_or_team="Player", pt_measure_type = "Possessions", per_mode_simple="PerGame")
    df = touch_data.get_data_frames()[0][[
            "PLAYER_ID", "PLAYER_NAME", "TOUCHES", "TIME_OF_POSS",
            "AVG_SEC_PER_TOUCH", "AVG_DRIB_PER_TOUCH", "PTS_PER_TOUCH"
        ]]

    player_stats_dict = {}

    for _, row in df.iterrows():
        player_stats_dict[row["PLAYER_NAME"]] = {
            "PLAYER_ID": row["PLAYER_ID"],
            "TOUCHES": row["TOUCHES"],
            "TIME_OF_POSS": row["TIME_OF_POSS"],
            "AVG_SEC_PER_TOUCH": row["AVG_SEC_PER_TOUCH"],
            "AVG_DRIB_PER_TOUCH": row["AVG_DRIB_PER_TOUCH"],
            "PTS_PER_TOUCH": row["PTS_PER_TOUCH"]
        }

    all_season_dicts[season] = player_stats_dict


2014-15
2015-16
2016-17
2017-18
2018-19
2019-20
2020-21
2021-22
2022-23
2023-24
2024-25


In [28]:
all_season_dicts

{'2014-15': {'AJ Price': {'PLAYER_ID': 201985,
   'TOUCHES': 30.3,
   'TIME_OF_POSS': 2.9,
   'AVG_SEC_PER_TOUCH': 5.67,
   'AVG_DRIB_PER_TOUCH': 5.92,
   'PTS_PER_TOUCH': 0.169},
  'Aaron Brooks': {'PLAYER_ID': 201166,
   'TOUCHES': 53.4,
   'TIME_OF_POSS': 4.8,
   'AVG_SEC_PER_TOUCH': 5.37,
   'AVG_DRIB_PER_TOUCH': 5.82,
   'PTS_PER_TOUCH': 0.218},
  'Aaron Gordon': {'PLAYER_ID': 203932,
   'TOUCHES': 22.4,
   'TIME_OF_POSS': 0.7,
   'AVG_SEC_PER_TOUCH': 1.94,
   'AVG_DRIB_PER_TOUCH': 1.17,
   'PTS_PER_TOUCH': 0.231},
  'Adreian Payne': {'PLAYER_ID': 203940,
   'TOUCHES': 39.3,
   'TIME_OF_POSS': 1.1,
   'AVG_SEC_PER_TOUCH': 1.73,
   'AVG_DRIB_PER_TOUCH': 0.44,
   'PTS_PER_TOUCH': 0.169},
  'Al Horford': {'PLAYER_ID': 201143,
   'TOUCHES': 58.2,
   'TIME_OF_POSS': 1.7,
   'AVG_SEC_PER_TOUCH': 1.7,
   'AVG_DRIB_PER_TOUCH': 0.59,
   'PTS_PER_TOUCH': 0.261},
  'Al Jefferson': {'PLAYER_ID': 2744,
   'TOUCHES': 39.8,
   'TIME_OF_POSS': 1.5,
   'AVG_SEC_PER_TOUCH': 2.29,
   'AVG_DRIB_PER_T

In [29]:
main_data_by_season = load_json_from_s3("all_stats_test.json")

In [37]:
from copy import deepcopy

# Optional: deep copy if you want to preserve the original data
merged_data = deepcopy(main_data_by_season)

for season, teams in merged_data.items():
    if season not in all_season_dicts:
        continue

    touch_stats = all_season_dicts[season]

    for team, players in teams.items():
        if isinstance(players, dict):
            for player_name, player_data in players.items():
                if player_name in touch_stats:
                    # Merge touch stats into player['stats']
                    player_data["stats"].update(touch_stats[player_name])


TypeError: Object of type Decimal is not JSON serializable

In [48]:
def transform_all_stats(data):
    for season, teams in data.items():
        for team, players in teams.items():
            key = f"{season}-{team}"
            yield {
                "id": key,
                "season": season,
                "team": team,
                "players": players
            }

In [52]:
def convert_floats_to_decimal(obj):
    if isinstance(obj, float):
        return Decimal(str(obj))  # Convert float to string first to avoid float precision issues
    elif isinstance(obj, dict):
        return {k: convert_floats_to_decimal(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [convert_floats_to_decimal(i) for i in obj]
    else:
        return obj

In [53]:
def upload_to_dynamodb(table_name, data):
    
    data = convert_floats_to_decimal(data)
    table = dynamodb.Table(table_name)
    count = 0
    for item in transform_all_stats(data):
        try:
            table.put_item(Item=item)
            count += 1
            if count % 50 == 0:
                print(f"...{count} items uploaded to {table_name}")
        except ClientError as e:
            print(f"❌ Error uploading to {table_name}: {e}")
    print(f"✅ Done uploading {count} items to {table_name}")

# Upload loop

In [54]:
upload_to_dynamodb('AllStats',merged_data)

...50 items uploaded to AllStats
...100 items uploaded to AllStats
...150 items uploaded to AllStats
...200 items uploaded to AllStats
...250 items uploaded to AllStats
...300 items uploaded to AllStats
...350 items uploaded to AllStats
✅ Done uploading 372 items to AllStats


In [40]:
merged_data["2024-25"]["NYK"]["Jalen Brunson"]["stats"]

{'PLAYER_ID': 1628973,
 'PLAYER_NAME': 'Jalen Brunson',
 'NICKNAME': 'Jalen',
 'TEAM_ID': 1610612752,
 'TEAM_ABBREVIATION': 'NYK',
 'AGE': Decimal('28.0'),
 'GP': 65,
 'W': 40,
 'L': 25,
 'W_PCT': Decimal('0.615'),
 'MIN': Decimal('35.4'),
 'FGM': Decimal('9.0'),
 'FGA': Decimal('18.5'),
 'FG_PCT': Decimal('0.488'),
 'FG3M': Decimal('2.3'),
 'FG3A': Decimal('6.1'),
 'FG3_PCT': Decimal('0.383'),
 'FTM': Decimal('5.7'),
 'FTA': Decimal('6.9'),
 'FT_PCT': Decimal('0.821'),
 'OREB': Decimal('0.4'),
 'DREB': Decimal('2.5'),
 'REB': Decimal('2.9'),
 'AST': Decimal('7.3'),
 'TOV': Decimal('2.5'),
 'STL': Decimal('0.9'),
 'BLK': Decimal('0.1'),
 'BLKA': Decimal('0.8'),
 'PF': Decimal('2.1'),
 'PFD': Decimal('6.3'),
 'PTS': Decimal('26.0'),
 'PLUS_MINUS': Decimal('2.2'),
 'NBA_FANTASY_PTS': Decimal('41.0'),
 'DD2': 14,
 'TD3': 0,
 'WNBA_FANTASY_PTS': Decimal('40.6'),
 'GP_RANK': 162,
 'W_RANK': 89,
 'L_RANK': 256,
 'W_PCT_RANK': 144,
 'MIN_RANK': 17,
 'FGM_RANK': 13,
 'FGA_RANK': 18,
 'FG_PCT_R